In [184]:
import os
import pandas as pd

In [192]:
data = ["CZ_ALBUMS_TOP_100_mesicni.csv","CZ_SINGLES_DIGITAL_TOP_50__CZ_tydeni.csv","CZ_RADIO_TOP_50_CZ_tydeni.csv"]

In [193]:
df = pd.DataFrame()
for f in data:
    df = pd.concat([df,pd.read_csv(os.path.join("data",f))])

## Nejdřív si obstaráme seznam tvůrců

In [194]:
interpreti = df.groupby("interpret_hlavni")['rok'].nunique().nlargest(5000).index.to_list()

In [195]:
len(interpreti)

964

In [189]:
interpreti

['Michal Hrůza',
 'Kryštof',
 'Kabát',
 'Lucie Bílá',
 'Lucie',
 'Support Lesbiens',
 'Chinaski',
 'Karel Gott',
 'Divokej Bill',
 'Anna K.',
 'No Name',
 'Miro Žbirka',
 'Daniel Landa',
 'Verona',
 'Dan Bárta',
 'Aneta Langerová',
 'Lucie Vondráčková',
 'Marek Ztracený',
 'Tomáš Klus',
 'O5',
 'Janek Ledecký',
 'Mandrage',
 'Ewa Farna',
 'Jaromír Nohavica',
 'Tereza Kerndlová',
 'Wohnout',
 'Xindl X',
 'Petr Kolář',
 'Richard Müller',
 'Mig 21',
 'Tři Sestry',
 'Škwor',
 'David Koller',
 'David Deyl',
 'Helena Vondráčková',
 'Ben Cristovao',
 'Olga Lounová',
 'Nightwork',
 'Kristina',
 'Ready Kirken',
 'Rybičky 48',
 'Čechomor',
 'Ego',
 'Václav Neckář',
 'Michal David',
 'Buty',
 'Voxel',
 'Jana Kirschner',
 'Petr Bende',
 'Hana Zagorová',
 'Marpo',
 'Lenny',
 'Wanastowi Vjecy',
 'Katarína Knechtová',
 'Debbi',
 'Jelen',
 'Slza',
 'Walda Gang',
 'Paulie Garand',
 'Eddie Stoilow',
 'Airfare',
 'Udg',
 'Clou',
 'Celeste Buckingham',
 'Majk Spirit',
 'Horkýže Slíže',
 'Adam Mišík',
 'Pe

### Zkusíme to automaticky

In [196]:
import requests

In [197]:
def get_id(query):
    API_ENDPOINT = "https://www.wikidata.org/w/api.php"
    params = {
        'action': 'wbsearchentities',
        'format': 'json',
        'language': 'en',
        'search': query
    }

    r = requests.get(API_ENDPOINT, params = params)
    
    try:
        
        return[r.json()['search'][0]['id'], r.json()['search'][0]['display']['label']['value'], r.json()['search'][0]['display']['description']['value']]
    
    except:
        return[None, None, None]

In [198]:
get_id('Chainsmokers')

[None, None, None]

In [199]:
from IPython.display import clear_output

In [200]:
qecka = {}
count = 0
for i in interpreti:
    print(f"""{count} / {len(interpreti)} / {i}""")
    qecka[i] = get_id(i)
    count = count + 1
    clear_output(wait=True)

963 / 964 / - Warner Music


In [201]:
if not os.path.exists("wikidata"):
    os.mkdir("wikidata")
else:
    pass

In [202]:
import json

In [203]:
with open(os.path.join("wikidata","wiki_raw.json"), "w+") as file:
    json.dump(qecka, file)

### Ručo doplnění kódů

In [204]:
for key, val in qecka.items():
    if val[2]:
        if not any(word in val[2] for word in ["singer", "band", "music", "rapper", "pop ","hip hop","orchest","group"]):
            print(f"{key}: {val}")

Kryštof: ['Q1770777', 'Kryštof', 'male given name']
Lucie: ['Q1617357', 'Lucie', 'female given name']
Anna K.: ['Q86723556', 'Anna K.', 'refrigerated cargo ship built in 1949']
Verona: ['Q2028', 'Verona', 'city in Veneto, northern Italy']
O5: ['Q6544459', 'lieutenant colonel', 'military rank of the United States']
Jaromír Nohavica: ['Q942232', 'Jaromír Nohavica', 'Czech songwriter, lyricist, and poet']
Petr Kolář: ['Q7178537', 'Petr Kolář', 'Czech diplomat']
Mig 21: ['Q150609', 'MiG-21', 'fighter aircraft family by Mikoyan']
Kristina: ['Q19798802', 'Kristina', 'female given name']
Ego: ['Q4426116', 'ego', 'self-identity in the psyche']
Voxel: ['Q663554', 'voxel', 'element representing a value on a grid in three dimensional space']
Lenny: ['Q1667498', 'Lenny', 'male given name']
Jelen: ['Q12024449', 'Jelen', 'family name']
Udg: ['Q1814503', 'University of Girona', 'public university based in Girona']
Clou: ['Q8074', 'cloud', 'visible mass of liquid droplets or frozen crystals suspended 

In [174]:
spravne = ['Jaromír Nohavica','Emma Smetana','Dara Rolins','Avicii','Deethane','Radim Hladík','Vojtěch Dyk','František Segrado','David Guetta','Bára Tlučhořová','Discoboys','Tiësto','Ty Nikdy','Jaroslav Svěcený','Jaroslav Uhlíř','Michal Horáček','Fattypillow','Pavel Šporcl','Přemek Forejt','Ondřej G. Brzobohatý']

In [205]:
for key, val in qecka.items():
    if not val[2]:
        print(f"{key}")

Rybičky 48
Walda Gang
Airfare
Doctor P.P.
Atmo Music
Medjmedj Sofian
Robin Zoot
Chainsmokers
Lenka Nová
Hodiny
Harlej
Linakaro
Stein27
The Silver Spoons
Czech It
Robin Mood
Žbirka Miro
Coco Jambo
Cocoman
Bek Ofis
Bílá Lucie
Grey256
Matej Smutný
Cristovao Ben
Óčko Allstars
Za100Let
Machálková Leona
Šinkorová D.
I.M.T.Smile
Michal Pelant
Langerová Aneta
Kurtizány Z 25. Avenue
Trautenberk
Vadak
Michael Burian
We On The Moon
Bůhví
The Carsleepers
Hard Rico
Fleret
Prokop Michal
E2 Crew
Suchý Jiří
Onemanshow Foundation
Pam Rabbit
Dj Boccelli
Flattus
Nedivoč
Cocotte Minute
Dollar Prync
Popcorn Drama
Prowizorium
Zoči Voči
Porsche Boy
Zdena Predná
Toneless
2Choice
Šmajda Miro
Refew
Rakovicky
Rolins Dara
Lidipop
Šenková Leona
Šafařík Daniel
Přenosilová Yvonne
Psycho Rhyme
Superstar 2009
Nová Lenka
Nosková Míša
Neřež
New Element
Nerez
Mikuláš Hrbáček
Tribbs
Napoleona
Tomáško Vojta "Kiďák"
One Night Band
Medvěd 009
Wolfhooker John
Wesome
Mark Dann
Ovoce
Vojtaano
Summer All Stars
Ro(C)K Podvraťáků-

In [173]:
opravy = {
    'Drake': 'Q33240',
    'Lisa': 'Q26707663',
    'Fiedlerski': 'Q114388633',
    'Kryštof': 'Q6774073',
    'Y2K': 'Q66937797',
    'Topic': 'Q20804322',
    'Script': 'Q459585',
    'Shouse': 'Q107478988',
    'Sigala': 'Q20973111',
    'Omi': 'Q18510281',
    'Koky': 'Q107782406',
    'Lucie': 'Q4963472',
    'Verona': 'Q3496586',
    'Anna K.': 'Q3508515',
    'O5': 'Q12041806',
    'Petr Kolář': 'Q12044915',
    'Mig 21': 'Q3062747',
    'Ego': 'Q12765182',
    'Kristina': 'Q430744',
    'Voxel':'Q19602849',
    'Lenny':'Q15249858',
    'Jelen':'Q19820296',
    
}

## Wikidata: the real shit

Chcu:
- žánr
- kapela/jednotlivec? (Ale bacha u interpret_hlavni!)
- země původu (co když víc?)
- rok narození
- místo narození
- rok úmrtí
- místo úmrtí
- rok začátku kariéry
Možná ještě:
- ocenění?

In [18]:
from wikidata.client import Client
import wikidata.entity

In [72]:
def wikidates(id):
    client = Client()
    entity = client.get(id, load=True)
    return entity

### Začátek a konec kariéry

In [95]:
wikidates('Q45188').attributes['claims']['P2031'][0]['mainsnak']['datavalue']['value']['time']

'+1997-00-00T00:00:00Z'

In [96]:
wikidates('Q45188').attributes['claims']['P2032'][0]['mainsnak']['datavalue']['value']['time']

KeyError: 'P2032'

work period (end) (P2032)

pohlavní identita subjektu; u lidí použijte muž (Q6581097), žena (Q6581072), transmuž (Q2449503), transžena (Q1052281); u zvířat použijte samec (Q44148), samice (Q43445); u skupin stejného pohlaví použijte „podtřída (čeho)“ (P279)

In [73]:
wikidates('Q45188').attributes['claims']

{'P358': [{'mainsnak': {'snaktype': 'value',
    'property': 'P358',
    'hash': '4cd91f2104436f2b24b0740042ca699db2f1cfc6',
    'datavalue': {'value': {'entity-type': 'item',
      'numeric-id': 547583,
      'id': 'Q547583'},
     'type': 'wikibase-entityid'},
    'datatype': 'wikibase-item'},
   'type': 'statement',
   'id': 'q45188$BD0D253C-AF10-4F3B-9671-238F964C8D08',
   'rank': 'normal',
   'references': [{'hash': 'fa278ebfc458360e5aed63d5058cca83c46134f1',
     'snaks': {'P143': [{'snaktype': 'value',
        'property': 'P143',
        'hash': 'e4f6d9441d0600513c4533c672b5ab472dc73694',
        'datavalue': {'value': {'entity-type': 'item',
          'numeric-id': 328,
          'id': 'Q328'},
         'type': 'wikibase-entityid'},
        'datatype': 'wikibase-item'}]},
     'snaks-order': ['P143']}]}],
 'P373': [{'mainsnak': {'snaktype': 'value',
    'property': 'P373',
    'hash': 'c93465ee1cdfb565aa39c69a7275c57b1069cef1',
    'datavalue': {'value': 'Coldplay', 'type': 'st

### Země původu

In [81]:
wikidates('Q45188').attributes['claims']['P495'][0]['mainsnak']['datavalue']['value']['id']

'Q145'

In [88]:
wikidates('Q145').attributes['claims']['P1705'][0]['mainsnak']['datavalue']['value']['text']

'United Kingdom of Great Britain and Northern Ireland'

In [42]:
def wikidates2(id):
    client = Client()
    entity = client.get(id, load=True)
    return dict(wikidata.entity.Entity("P551", client))

In [43]:
wikidates2('Q33240')

{<wikidata.entity.Entity P31>: <wikidata.entity.Entity Q18608871>,
 <wikidata.entity.Entity P1659>: <wikidata.entity.Entity P937>,
 <wikidata.entity.Entity P1855>: <wikidata.entity.Entity Q60543>,
 <wikidata.entity.Entity P2875>: <wikidata.entity.Entity Q23909040>,
 <wikidata.entity.Entity P1628>: 'https://schema.org/homeLocation',
 <wikidata.entity.Entity P3254>: 'https://www.wikidata.org/wiki/Wikidata:Property_proposal/Archive/7#P551',
 <wikidata.entity.Entity P2302>: <wikidata.entity.Entity Q21510865>,
 <wikidata.entity.Entity P1647>: <wikidata.entity.Entity P276>,
 <wikidata.entity.Entity P1629>: <wikidata.entity.Entity Q699405>,
 <wikidata.entity.Entity P3734>: <wikidata.entity.Entity Q42533339>,
 <wikidata.entity.Entity P6609>: <wikidata.entity.Entity P131>,
 <wikidata.entity.Entity P8274>: <wikidata.entity.Entity Q44601380>,
 <wikidata.entity.Entity P7087>: <wikidata.entity.Entity Q107616534>,
 <wikidata.entity.Entity P2668>: <wikidata.entity.Entity Q24025284>}

In [65]:
def get_gender(wikidata_id):
    client = Client()  
    
    entity = client.get(wikidata_id)  # Retrieve the entity using the Wikidata ID
        
    print(entity.attributes['claims']['P21'][0]['mainsnak']['datavalue']['value']['id'])

In [67]:
get_gender('Q45188')

KeyError: 'P21'